# CrewAI

How to use:
- Run "Setup"
- Jump to one of the chapters and execute the cells, all chapters are independent from each other

# TOC

1. [CrewAI: Tool Tests](#CrewAI:-Tool-Tests)
1. [Web Researcher](#1.-Web-Researcher)

# Notes

## Tools

- ServerDevTool (Google) works better than DuckDuckGoSearchRun (DuckDuckGo). Eg. it provides the dates of the found articles, which DuckDuckGoSearchRun does not do

# Setup

In [ ]:
#!uv pip install jupyterlab==4.2.1 pyautogen==0.2.27 crewai==0.41.1 crewai-tools==0.4.26 duckduckgo-search==6.1.4

In [ ]:
#!uv pip install agentops==0.3.4
#!uv pip install langtrace-python-sdk

In [ ]:
#uv pip freeze > requirements2.txt
#uv pip install -r requirements.txt

In [1]:
# Warning control
import warnings
#warnings.simplefilter("ignore", category=Warning)
warnings.filterwarnings("ignore")

In [7]:
from IPython.display import Markdown,display
import json
import os
import shutil
import psutil
from getpass import getpass
import openai
from os import environ

IN_NOTEBOOK = any(["jupyter-lab" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

if environ.get('OPENAI_API_KEY') is None:
    print('Environment variable not found. Setting values...')
    os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential'] 
    os.environ["TOGETHERAI_API_KEY"] = CREDS['together-ai']['key']['credential']
    os.environ['ANTHROPIC_API_KEY'] = CREDS['anthropic']['key']['credential']
    os.environ["SERPER_API_KEY"] = CREDS['serper_dev']['key']['credential']
    #openai.api_key = CREDS['OpenAI']['v2']['credential'] 

Secrets (JSON string):  ········


Environment variable not found. Setting values...


In [4]:
# helpers

def delete_directories(dirs):
    # deletes array of directories
    for d in dirs:
        if os.path.exists(d):
            shutil.rmtree(d)
            display(d + ' deleted')

# Activate tracing (best: AgentOps)

In [ ]:
# Enable Agentops Tracing
# uv pip install agentops==0.3.4
os.environ["AGENTOPS_API_KEY"] = CREDS['agentops']['key']['credential']
import agentops
agentops.init()

In [ ]:
# Enable Langchain Tracing
#os.environ["LANGCHAIN_API_KEY"] = CREDS['langsmith']['key']['credential']
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
# Enable Langtrace Tracing
# via Docker Compose Container => https://docs.langtrace.ai/hosting/hosting_options/docker-compose
#from langtrace_python_sdk import langtrace
#langtrace.init(
#  api_key=CREDS['langtrace']['key']['credential'],
#  api_host="http://localhost:3000/api/trace",
#)

In [ ]:
# Get list of available models
client = openai.OpenAI()
model_list = client.models.list()
for model in model_list:
  print(model.id)

**Test OpenAI Chat and Langtrace**

In [ ]:
client = openai.OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages = [{"role": "user", "content": "this is a test request, write a short poem"}]
)
print(response.choices[0].message.content)

In [ ]:
# Langtrace test with root-span annotation

from langtrace_python_sdk.utils.with_root_span import with_langtrace_root_span

@with_langtrace_root_span()
def example():
    client = openai
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "How many states of matter are there?"
            }
        ],
    )
    print(response.choices[0].message.content)

example()

# CrewAI Default model

In [5]:
os.environ["OPENAI_MODEL_NAME"]="gpt-4o"
#os.environ["OPENAI_MODEL_NAME"]="gpt-3.5-turbo"

# CrewAI - Hello World (in 3 coding style variants)

- Simple crew with 1 agent to write a report about the opening of the Olympic games 2024 in Munich (!)
- Without tools, the agent hallucinates about the opening in Munich
- With the tool `search_tool_duckduckgo` it queries the web and report about the opening in Paris
- By using `%%capture` all logging information will be supressed. It can be shown by using `myoutput.show()`

## Variant 1: Vanilla code

In [ ]:
%%capture --no-display myoutput 
import datetime
from crewai import Agent, Crew, Process, Task
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI

search_tool_duckduckgo = DuckDuckGoSearchRun()

researcher = Agent(
    role='Senior Researcher',
    goal='Answer the following question: {topic}',
    verbose=True,
    backstory=("Driven by curiosity, you're at the forefront of information research, eager to explore and find knowledge."),
    #llm=ChatOpenAI(model_name="gpt-4", temperature=0.8),
    #tools= [search_tool_duckduckgo] # comment / uncomment to use search tool
)

do_research = Task(
    description=("Answer the question '{topic}'. Answer in some short paragraphs, do not add anything else."),
    expected_output="The answer to the question '{topic}'",
    agent=researcher, 
    output_file=f'output/{datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")} vanilla code.txt',
    create_directory=True
)

crew = Crew(
    agents=[researcher],
    tasks=[do_research],
    cache=False,
    verbose=2
)
result = crew.kickoff(inputs={'topic': 'Write obout the opening of the Olympic games 2024 in Munich.'}) 
display('Done.')
#myoutput.show() # show all log information, stored in variable myoutput
display(Markdown(result.raw))


## Variant 2: YAML configuration

The same agents and tasks as above, but configured via YAML-string / -file

In [ ]:
%%capture --no-display myoutput 
os.environ["OPENAI_MODEL_NAME"]="gpt-3.5-turbo"
from crewai import Agent, Crew, Process, Task
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI
import yaml

search_tool_duckduckgo = DuckDuckGoSearchRun()

yamlAgents = '''
researcher:
  role: >
    Senior Researcher
  goal: >
    Answer the following question: {topic}
  backstory: >
    Driven by curiosity, you're at the forefront of information research, eager to explore and find knowledge.
'''

yamlTasks = '''
do_research:
  description: >
    Answer the question '{topic}'. Answer in some short paragraphs, do not add anything else.
  expected_output: >
    The answer to the question '{topic}'
'''

agents_config = yaml.safe_load(yamlAgents)
tasks_config = yaml.safe_load(yamlTasks)

#with open('./config-hello-world/agents.yaml', 'r') as file:
#    agents_config = yaml.safe_load(file)
#with open('./config-hello-world/tasks.yaml', 'r') as file:
#    tasks_config = yaml.safe_load(file)

researcher = Agent(
    config=agents_config['researcher'], 
    verbose=True,
    #llm=ChatOpenAI(model_name="gpt-4", temperature=0.8),
    #tools= [search_tool_duckduckgo] # comment / uncomment to use search tool
)

do_research = Task(
    description=tasks_config["do_research"]["description"],
    expected_output=tasks_config["do_research"]["expected_output"],
    agent=researcher,
    output_file=f'output/{datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")} yaml code.txt',
    create_directory=True
)

crew = Crew(
  agents=[researcher],
  tasks=[do_research],
  cache=False,
  verbose=2
)
result = crew.kickoff(inputs={'topic': 'Write obout the opening of the Olympic games 2024 in Munich.'}) 
display('Done.')
#myoutput.show() # show all log information, stored in variable myoutput
display(Markdown(result.raw))

## Variant 3: Annotations and YAML configuration

The same agents and tasks as above, but configured via YAML-file.

Works only with YAML-files, not with YAML-strings.

**Requirement:** Files 'agents.yaml' and 'tasks.yaml' (see Variant 2 above) available in directory 'config-hello-world'

# Execute 2 tasks in parallel/asynchronously, create JSON output for a task

- Two web research tasks are started in parallel to save time. 
- The results of one research tasks are providei in JSON output:
    - Define model (via Pydantic)
    - Assign model to list_news task `output_json`
    - Access JSON via `list_news.output.json_dict`

In [ ]:
%%capture --no-display myoutput 
from crewai import Agent, Crew, Process, Task
from crewai_tools import SerperDevTool
from pydantic import BaseModel, Field
from typing import List, Optional

search_tool_serper = SerperDevTool()

research_agent = Agent(
  role='Researcher',
  goal='Find and summarize the latest AI news for developers',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights to the business.""",
  verbose=True
)

class AISingleNewsModel(BaseModel):
	"""AI single news model"""
	title: str = Field(..., description="Title of the news")
	url: str = Field(..., description="Source URL of the news")

class AINewsListModel(BaseModel):
	"""AI news list model"""
	news: List[AISingleNewsModel] = Field(..., description="List of AI news")

list_news = Task(
    description="List of 5 latest developer news in 2024 for an article about AI.",
    expected_output="A bullet list summary of the top 5 most important AI news from techradar.com. Add the Source-URL as readable text at the end of each bullet point.",
    agent=research_agent,
    tools=[search_tool_serper],
    output_json=AINewsListModel, # JSON output
    async_execution=True # Will be executed asynchronously
)

list_important_history = Task(
    description="Research the history of AI and give me the 5 most important events.",
    expected_output="Bullet point list of 5 important historical events.",
    agent=research_agent,
    tools=[search_tool_serper],
    async_execution=True # Will be executed asynchronously
)

write_article = Task(
    description="Write an article about the history of AI and the latest news from techradar.com for AI developers in 2024.",
    expected_output="A 5 paragraph article about AI, formatted in markdown syntax.",
    agent=research_agent,
    context=[list_news, list_important_history] # Will wait for the output of the two tasks to be completed
)

crew = Crew(
    agents=[research_agent],
    tasks=[list_news, list_important_history, write_article],
    cache=False,
    verbose=2
)

result = crew.kickoff() 
display('Done.')
#myoutput.show() # show all log information, stored in variable myoutput
display(Markdown(result.raw))
display('')
display('')
display(Markdown(f"JSON Output: {json.dumps(list_news.output.json_dict, indent=2)}"))


In [ ]:
# Accessing the task output
task_output = list_news.output

print(f"Task Description: {task_output.description}")
print(f"Task Summary: {task_output.summary}")
print(f"Raw Output: {task_output.raw}")
if task_output.json_dict:
    print(f"JSON Output: {json.dumps(task_output.json_dict, indent=2)}")
if task_output.pydantic:
    print(f"Pydantic Output: {task_output.pydantic}")

# Tools

## Website Search Tool (RAG)

- prompt engineering is really important. Be very precise: "Visit all 5 news articles by using the source URL..."
- under development, buggy
- there seems to be a cache mechanism (?): Searches sometimes deliver wrong results (from the cache).
- **restart kernel - this fixes most of the issues!**

In [6]:
#%%capture --no-display myoutput 

delete_directories(['db'])

import datetime
from crewai import Agent, Crew, Process, Task
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)
import datetime

search_tool_serper = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

research_agent = Agent(
  role='Researcher',
  goal='Find and summarize the latest AI news for developers',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights to the business.""",
  verbose=True
)

list_news = Task(
    description="Get a list of the 5 newest developer news in 2024 for an article about AI.",
    expected_output="A bullet list summary of the 5 newest AI news from techradar.com. Add the source URL as readable text at the end of each bullet point.",
    agent=research_agent,
    tools=[search_tool_serper]
)

get_news_details = Task(
    description="Visit all 5 news articles by using the source URL and get all the details of their content to summarize them.", # by using the web_rag_tool 
    expected_output="A summary of all 5 news articles, each article is summarized by 10 bullet points.",
    agent=research_agent,
    tools=[web_rag_tool],
    #context=[list_news]
)

write_article = Task(
    description="Write an article about the latest 5 news from techradar.com for AI developers in 2024.",
    expected_output="A detailed article about current AI news for developers. Each news should consist of 5 paragraphs, formatted in markdown syntax.",
    agent=research_agent,
    output_file=f'output/{datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")} 5 top AI news blogpost.txt',
    create_directory=True,
    #context=[get_news_details]
)

crew = Crew(
    agents=[research_agent],
    tasks=[list_news, get_news_details, write_article],
    #process=Process.sequential,
    memory=True,
    cache=False,
    verbose=2
)

result = crew.kickoff() 
display('Done.')
#myoutput.show() # show all log information, stored in variable myoutput
display(Markdown(result.raw))


 [2024-07-27 21:52:24][DEBUG]: == Working Agent: Researcher
 [2024-07-27 21:52:24][INFO]: == Starting Task: Get a list of the 5 newest developer news in 2024 for an article about AI.


> Entering new CrewAgentExecutor chain...
To find the 5 newest AI news for developers from TechRadar in 2024, I need to perform an internet search specifically targeting TechRadar's latest AI articles for developers in the year 2024.

Action: Search the internet
Action Input: {"search_query": "latest AI news for developers site:techradar.com 2024"} 


Search results: Title: Developers aren't worried about AI taking their jobs
Link: https://www.techradar.com/pro/developers-arent-worried-about-ai-taking-their-jobs-but-excited-about-productivity-gains
Snippet: Developers aren't worried about AI taking their jobs — but excited about productivity gains. Workers are increasingly adopting artificial ...
---
Title: Mark Zuckerberg claims open-source is 'necessary for a ...
Link: https://www.techradar.com/computi

Inserting batches in chromadb: 100%|██████████████| 1/1 [00:00<00:00,  1.91it/s]


 

Relevant Content:
Developers aren't worried about AI taking their jobs — but excited about productivity gains | TechRadar Skip to main content (Image credit: Getty Images) Workers are increasingly adopting artificial intelligence tools, with three in four (76%) now using AI in their daily work, new research has claimed.Stack Overflow has shared a detailed analysis of the current state of artificial intelligence use across the industry in its latest Develop Survey, which takes into account responses from more than 65,000 developers across 185 countries, making this one of the most detailed and extensive studies of its kind.However, the study serves as a reminder that AI’s primary benefit is as a supportive tool to human workers, rather than a total replacement.AI will help developers, and not replace themThe survey found only two in five (43%) trust the accuracy of AI – a minor increase of one percentage point compared with last year’s study. The report also touches on AI’s inability

Inserting batches in chromadb: 100%|██████████████| 1/1 [00:00<00:00,  2.21it/s]


 

Relevant Content:
Mark Zuckerberg claims open-source is 'necessary for a positive AI future' | TechRadar Skip to main content (Image credit: Meta) Meta CEO Mark Zuckerberg called for an open-source future for generative AI in a new essay. He argues that an open-source ecosystem benefits consumers globally and encourages innovation, indirectly throwing a rhetorical gauntlet at the feet of closed-source developers (and Meta competitors) like OpenAI and Google. “Meta’s business model is about building the best experiences and services for people,” Zuckerberg wrote. “To do this, we must ensure that we always have access to the best technology, and that we’re not locking into a competitor’s closed ecosystem where they can restrict what we build.”The principle behind open-source models is that they allow for a collaborative approach to development. Unlike closed-source models, which are restricted to a company’s internal team and its customers, open-source models can be accessed, modified

...

 

Relevant Content:
Google I/O 2024 – the 7 biggest AI announcements, from Gemini to Android 15 | TechRadar Skip to main content (Image credit: Google / Future) The dust has now settled on the Google I/O 2024 keynote and there's no doubting what the big theme was – Google Gemini and new AI tools completely dominated the announcements, giving us a glimpse of where our digital lives are headed. CEO Sundar Pichai was right to describe the event as its version of The Eras Tour – specifically, the "Gemini Era" – at the very top. Unlike previous years, the entire keynote was about Gemini and AI; in fact, Google said the latter a total of 121 times. From unveiling a futuristic AI assistant called "Project Astra" that can run on a phone – and maybe smart glasses, one day – to Gemini being infused into nearly every service or product the company offers, artificial intelligence was definitely the dominant theme. The two-hour keynote was enough to melt the mind of all but the most ardent LLM ent

Inserting batches in chromadb: 100%|██████████████| 1/1 [00:00<00:00,  1.09it/s]


 

Relevant Content:
Will Apple go big on AI at WWDC 2024? Almost certainly – but it could ‘think different’ | TechRadar Skip to main content (Image credit: Shutterstock / Yuganov Konstantin) You can’t swing a metaphorical cat in a virtual room without some tech-savvy person harping on about AI. And over the past year and a bit, AI interest has exploded thanks to the rise of generative AI; for the uninitiated, that’s smart algorithms and modes that can basically create things from various commands and inputs rather than acting as a smart assistant. As such, we saw the Google Pixel 8 and Pixel 8 Pro go hard on generative AI, specifically with its Magic Editor and smart features that can pretty much handle calls for you. Samsung then did the same with the Galaxy S24 series, on which the rather snazzy Circle to Search AI feature made its debut. Google I/O was a feast of AI announcements and forward-thinking, with Gemini AI in the limelight. Here's how to watch WWDC 2024So that leads me to

'Done.'

my best complete final answer to the task.

---

## The Latest AI News for Developers in 2024

Artificial Intelligence continues to revolutionize the tech industry, and developers are at the forefront of this transformation. From productivity gains to open-source advocacy, the latest developments in AI are shaping how technology evolves. Here are the top five AI news stories for developers in 2024.

### 1. Developers Aren't Worried About AI Taking Their Jobs — But Excited About Productivity Gains

Recent research indicates that three out of four (76%) workers are now integrating AI tools into their daily workflows. According to Stack Overflow's Developer Survey, which surveyed over 65,000 developers from 185 countries, AI is seen more as a supportive tool than a job threat. Despite the growing use of AI, only 43% of developers completely trust its accuracy, a slight increase from last year's findings.

While AI struggles with more complex tasks, developers are optimistic about its impact on productivity. A significant 81% of developers reported increased productivity as the primary benefit of AI, with 71% of new coders finding that AI accelerates their learning process. Stack Overflow's Chief Product Officer, Ryan Polk, emphasized that while Generative AI (GenAI) is powerful, it is not a one-size-fits-all solution for technological challenges.

However, concerns remain prevalent, with 79% of developers worried about AI spreading misinformation, 65% about incorrect data attribution, and 50% about bias. Despite these concerns, 70% of professional developers do not see AI as a threat to their job security. Companies are encouraged to consider these perceptions to better integrate AI into workflows and enhance productivity.

Craig Hale, a tech and automotive writer with a keen interest in AI and machine learning, authored the article, highlighting the balance between AI's potential benefits and its current limitations. As AI continues to evolve, developers' insights will be crucial in navigating its integration into various industries.

### 2. Mark Zuckerberg Claims Open-Source Is 'Necessary for a Positive AI Future'

Meta CEO Mark Zuckerberg has taken a firm stance on the necessity of open-source models for the future of generative AI. In his advocacy, Zuckerberg criticizes closed-source developers like OpenAI and Google for restricting technological access, which he believes hampers innovation. He argues that open-source models foster collaborative development, unlike their closed-source counterparts, which are limited to internal teams and select customers.

Zuckerberg draws a comparison between open-source Android and Apple's proprietary iOS to illustrate the accessibility differences. Closed-source AI models, such as those from OpenAI and Google, offer limited opportunities for modification, thereby protecting their intellectual property. However, Zuckerberg contends that an open-source framework allows for faster and more effective refinement of AI technologies.

Acknowledging the risks of malicious use, Zuckerberg nevertheless emphasizes that closed-source models are not immune to such threats. Meta's commitment to open-source is evident through its Llama AI model portfolio, with Llama 3.1 being one of the most advanced iterations. By making these models open-source, Zuckerberg believes that a broader developer base can contribute, enhancing robustness and security.

The essay concludes that open-source AI democratizes development, ensuring that the benefits are widely distributed rather than concentrated in a few companies. This approach is seen as essential for a positive AI future, fostering innovation and collaboration across the global tech community.

### 3. OpenAI May Be Working on AI That Can Perform Research Without Human Help

OpenAI is reportedly developing a groundbreaking project named "Strawberry," aimed at enhancing AI models' reasoning capabilities. The primary goal of Strawberry is to create AI that can independently conduct follow-up research, moving beyond the current models' dependency on existing datasets. This involves a specialized post-training processing method on extensive datasets, setting new benchmarks in AI development.

Strawberry has shown promise in handling complex science and math problems that current commercial models cannot. Internal documents from OpenAI reveal that Strawberry, initially known as Q*, has achieved significant breakthroughs, including scoring over 90% on championship-level math problem datasets. If successful, Strawberry could revolutionize scientific research and problem-solving by autonomously filling knowledge gaps and proposing hypotheses.

This project aligns with OpenAI's long-term vision of achieving artificial general intelligence (AGI). Despite the advancements seen with GPT-3 and GPT-4, Strawberry represents a significant leap toward autonomous reasoning and deep research capabilities. The development of Strawberry could mark a pivotal moment in AI research, bringing us closer to truly autonomous AI systems capable of independent scientific inquiry.

### 4. Google I/O 2024 – The 7 Biggest AI Announcements

Google I/O 2024 was a landmark event, with CEO Sundar Pichai declaring it the "Gemini Era" due to the focus on AI advancements. The keynote featured significant announcements, primarily centered around Google Gemini and new AI tools. One of the standout introductions was Project Astra, an "AI agent" designed to understand, reason, and respond to live video and audio inputs.

Project Astra was demonstrated using a Pixel phone, showcasing its ability to interact with live feeds and provide real-time answers to questions. This AI agent represents a significant step forward in making digital experiences more interactive and intuitive. Additionally, Gemini's integration across nearly all Google services and products was a key highlight, emphasizing its multi-modal and long-context capabilities.

The keynote also underscored the potential of AI tools like Project Astra to enhance everyday digital interactions. From smartphones to potential smart glasses, the versatility and power of Gemini were evident. Google's focus on AI at the event reflects its strategic direction towards deeper AI integration within its ecosystem. These announcements set the stage for future developments in AI and their impact on consumer technology.

### 5. Will Apple Go Big on AI at WWDC 2024?

The surge in interest around generative AI, which creates content from various inputs, has led to speculation about Apple's plans for WWDC 2024. While Google and Samsung have already integrated generative AI into their products, Apple's recent hardware updates suggest a significant focus on AI. The new iPad Air and OLED iPad Pro models with M4 chips are designed for AI workloads, hinting at potential AI-centric announcements.

Apple's approach to AI is expected to be consumer-focused, seamlessly integrating generative AI into daily use without making it feel like a separate entity. There is speculation that Apple might either develop in-house AI models or partner with external developers like OpenAI or Google. Historically, Apple has integrated machine learning into its devices, such as using AI for photo enhancements in iPhones.

Potential AI features at WWDC 2024 could include intelligent notifications, auto-editing for media, customized fitness plans, and smart playlists. Siri is also expected to receive significant upgrades, possibly integrating with ChatGPT to compete with Alexa and Google Assistant. Apple's generative AI tools are likely to enhance user experiences naturally, reflecting the company's strategic direction in AI technology. The upcoming WWDC 2024 is anticipated to be a pivotal event for AI advancements, reinforcing Apple's commitment to integrating AI seamlessly into its ecosystem.

---

This detailed article encapsulates the latest AI news for developers, providing insights into the ongoing developments and their implications for the tech industry.

# From here: Tested with CrewAI 0.30.11

# CrewAI: Tool Tests: Web search, files

- Simple crew just for testing some tools
- The crew is created by a function `createCrew(mytools, myverbosity)` which makes it easier to check different tools with the same crew

In [ ]:
from crewai import Agent, Task, Crew, Process

os.environ["OPENAI_MODEL_NAME"]= "gpt-3.5-turbo" # or "gpt-4o" # or "gpt-4-1106-preview"

# file tools
from crewai_tools import DirectoryReadTool, FileReadTool, BaseTool
directory_read_tool = DirectoryReadTool(directory='./campaign-instructions')
file_read_tool = FileReadTool()

class WriteFileTool(BaseTool):
    name: str ="Write File Tool"
    description: str = ("""Useful to write a file to a given path with a given content. 
       The input to this tool should be a pipe (|) separated text 
       of length two, representing the full path of the file, 
       and the text content you want to write to it.
       For example, `chapter01.txt|CONTENT_PLACEHOLDER`.
       Replace CONTENT_PLACEHOLDER with the actual 
       text content you want to write to the file.""")
    def _run(self, data: str) -> str:
        try:
          path, content = data.split("|")
          path = path.replace("\n", "").replace(" ", "").replace("`", "")
          if not path.startswith("./workdir"):
            path = f"./workdir/{path}"
          with open(path, "w") as f:
            f.write(content)
          return f"File written to {path}."
        except Exception:
          return "Error with the input format for the tool."
write_file_tool1 = WriteFileTool()


# online search tools
from crewai_tools import SerperDevTool
from langchain_community.tools import DuckDuckGoSearchRun
search_tool_google = SerperDevTool()
search_tool_duckduckgo = DuckDuckGoSearchRun()
search_tool = search_tool_google

# Creating a crew, tools can be passed by argument
def createCrew(mytools, myverbosity):
    researcher = Agent(
      role='Senior Researcher',
      goal='Answer the following question: {topic}',
      verbose=True if myverbosity == True else False,
      memory=True,
      backstory=(
        "Driven by curiosity, you're at the forefront of"
        "information research, eager to explore and find knowledge."
      ),
      tools= mytools
      #allow_delegation=True
    )

    answer = Task(
      description=(
        "Answer the question '{topic}'."
        "Answer in a short and precise way, do not add anything else." # Use bulletpoints to structure your answer. "
        "Write your answer to a file."
      ),
      expected_output="The answer to the question '{topic}'", #,formatted as markdown.
      agent=researcher,
      output_file='agent-output.txt'  # This is necessary to create the output as file
    )

    # Forming the tech-focused crew with some enhanced configurations
    crew = Crew(
      agents=[researcher],
      tasks=[answer],
      memory=True,
      cache=False,
      max_rpm=100,
      verbose=2 if myverbosity == True else 0,
    )
    return crew

In [ ]:
crew = createCrew([directory_read_tool, file_read_tool], False)
result = crew.kickoff(inputs={'topic': 'What keypoints should be adressed for enterprise solutions'}) 
print(result)

In [ ]:
crew = createCrew([write_file_tool1], True)
result = crew.kickoff(inputs={'topic': 'Write three sentences about Harry Potter and write the text to a file named "harry.txt"'}) # with a corresponding name
print(result)

In [ ]:
crew = createCrew([], False)
result = crew.kickoff(inputs={'topic': 'What is 10+20?'}) 
print(result)

In [ ]:
from IPython.display import Markdown,display
display(Markdown(result.raw))

# Tools - work in progress

File write with langchain tools. Throws error message:

    This was the error: 1 validation error for WriteFileInput
    text
    field required (type=value_error.missing).

In [ ]:
# file write: see https://github.com/joaomdmoura/crewAI-examples/blob/main/landing_page_generator/tools/file_tools.py
# langchain file tools: https://python.langchain.com/v0.2/docs/integrations/tools/filesystem/#the-filemanagementtoolkit
# https://github.com/langchain-ai/langchain/issues/5226
from langchain.agents import Tool
from langchain.agents.agent_toolkits import FileManagementToolkit
fileToolkit = FileManagementToolkit(root_dir='workdir')
write_file_from_fileToolkit = fileToolkit.get_tools()[5] # there are multiple tools inside, #5 is write file
write_file_tool2 = Tool(
  name="Write text to directory",
  description="Useful to write text to a directory.",
  func=write_file_from_fileToolkit.run,
)
crew = createCrew([write_file_tool2], True)
result = crew.kickoff(inputs={'topic': 'Write three sentences about Harry Potter and write the text to a file named "harry.txt"'}) # with a corresponding name
print(result)

# 1. Web Researcher 

- searches Google and returns Top 5 results report
- adopted from example below

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_community.tools import DuckDuckGoSearchRun

search_tool_google = SerperDevTool()
search_tool_duckduckgo = DuckDuckGoSearchRun()

search_tool = search_tool_google

os.environ["OPENAI_MODEL_NAME"]="gpt-3.5-turbo" # or "gpt-4o" # or "gpt-4-1106-preview"

# Creating a senior researcher agent
researcher = Agent(
  role='Senior Researcher',
  goal='Find most relevant sources in the web {topic}',
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "information research, eager to explore and find knowledge."
  ),
  tools=[search_tool],
  allow_delegation=False,
	verbose=True
)

# Creating a writer agent with 
writer = Agent(
  role='Writer',
  goal='Writing precise reports about {topic}',
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "reports and summaries on an expert level."
  ),
  allow_delegation=False,
  verbose=True
)

In [ ]:
# Research task
research_task = Task(
  description=(
    "Search the web for the most relevant articles about {topic} in 2024."
    "Focus on sources like scientific papers, newspapers and expert blogs."
    "Your final report should list the 5 most relevant articles including URL, date, headline and a short summary."
  ),
  expected_output='A comprehensive list of search results.',
  tools=[search_tool],
  agent=researcher
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful report on {topic}."
    "Take the given articles and write for each of them one chapter. Include the source URL and date."
  ),
  expected_output='A 4 report on {topic}, listing the most relevant sources, formatted as markdown.',
  #tools=[search_tool_google],
  agent=writer
)

In [ ]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

In [ ]:
# Starting the task execution process with enhanced feedback
# result = crew.kickoff(inputs={'topic': 'will donald trumps chances of winning the presidential election diminish after his conviction in court?'})
# result = crew.kickoff(inputs={'topic': 'Will Apples Siri get smarter in 2024?'}) 
result = crew.kickoff(inputs={'topic': 'Will Apples Siri get smarter in 2024? Answer in German language'}) 
print(result)

In [ ]:
from IPython.display import Markdown,display
display(Markdown(result.raw))

# 2. Blog post writer: Researcher, Writer

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

os.environ["OPENAI_MODEL_NAME"]="gpt-4o" # or "gpt-4-1106-preview"

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

In [ ]:
# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

In [ ]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

In [ ]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'}) # in healthcare # in software development
print(result)

In [ ]:
print(result)

# 3. Blog post writer: Planner, Writer, Editor

Blog post writer Multi-Agent System using Crewai and Ollama

Source: https://medium.com/the-ai-forum/create-a-blog-writer-multi-agent-system-using-crewai-and-ollama-f47654a5e1cd

In [ ]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model = "crewai-llama3",
    base_url = "http://localhost:11434/v1")

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic} in 'https://medium.com/'."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "You have to prepare a detailed "
              "outline and the relevant topics and sub-topics that has to be a part of the"
              "blogpost."
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    #llm=llm,
    allow_delegation=False,
 verbose=True
)
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic} in 'https://medium.com/'. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    #llm=llm,
    verbose=True
)
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization 'https://medium.com/'. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    #llm=llm,
    allow_delegation=False,
    verbose=True
)

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
  "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)
inputs = {"topic":"Comparative study of LangGraph, Autogen and Crewai for building multi-agent system."}
result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown,display
display(Markdown(result))

# 4. Poem Writer: Manager, Researcher, Writer

Source: https://medium.com/thedeephub/crafting-custom-love-poems-with-crewai-agents-5ee3051b4414

In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain.tools import tool
from crewai.tasks.task_output import TaskOutput
from langchain_openai import ChatOpenAI

In [ ]:
# Define the topic 
# In this case, since we want a poem for our loved one, the topic should be things that they like and that you want to include in the poem
topic = "Taylor Swift and long walks on the beach at sunset"

In [ ]:
# Define the search tool 
search_tool = DuckDuckGoSearchRun()

# Function that is run after the task is completed
def callback_function(output: TaskOutput):
    ## What to do after the task is completed (i.e. send an sms with a love poem)
    print(f"""
        Task completed with the following output:
        Task: {output.description}
        Output: {output.result}
    """)

# Custom tool - read webpage
class ContentTools:
    @tool("Read webpage content")
    def read_content(url: str) -> str:
        """Read the content of a webpage"""
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        text_content = soup.get_text()
        return text_content[:5000]

In [ ]:
# First agent: the love poem manager
manager = Agent(
    role = "Love Poem Manager",
    goal = "Manage the process of creating a love poem for the partner of the client",
    verbose = True,
    backstory = "With a mind for romance, I want to deliver the best custom love poem to my client to give to their partner for Valentines Day",
    allow_delegation = True,
    max_iter = 10,
    max_rpm = 20,
)
# Second agent: the love poem searcher
researcher = Agent(
    role = "Love Poem Searcher",
    goal = f"Find a love poem for my partner, who really likes {topic}",
    verbose = True,
    backstory = """You are a curious love poem searcher, and you want to 
    find the best and most custom love poem for your client to give to their partner, 
    therefore you need some details about the things that the partner really likes! You will search for these details.""",
)
# Third agent: the love poem writer
writer = Agent(
    role = "Love Poem Writer",
    goal = f"Write a love poem for my partner, who really likes {topic}",
    verbose = True,
    backstory = """You are a creative love poem writer, and you want to 
    write the best and most custom love poem for your client to give to their partner. 
    With a mind for romance and a deep love for poetry, you find the perfect poetry style 
    for your client and create a wonderful poem that conveys love, respect, kindness, and admiration""",
)

In [ ]:
# Task 1: List ideas for a love poem
list_ideas = Task(
    description = "List of 5 cute ideas for a love poem, including possible poetry styles",
    expected_output = "Bullet point of 5 cute ideas for a love poem",
    tools=[search_tool, ContentTools().read_content],
    agent = researcher,
    async_execution=True,
)

# Task 2: List important facts related to topic
list_facts = Task(
    description = f"List of 5 important facts specifically for someone that likes {topic}",
    expected_output = "Bullet point of 5 important facts related to the given topic for a love poem",
    tools=[search_tool, ContentTools().read_content],
    agent = researcher,
    async_execution=True,
)

# Task 3: Write the love poem
write_poem = Task(
    description = f"Write a poem that would make Romeo and Juliet blush. The poem should be custom and tailored to the client's partner, who really likes {topic}",
    expected_output = "A lovely poem in a style that the client would like",
    tools=[search_tool],
    agent = writer,
    context=[list_ideas, list_facts],
    callback=callback_function,
)

# Task 4: Coordinate the love poem
manager_task = Task(
    description=f"""Oversee the creation of a great love poem for the client, 
    a lovebird who's looking for a custom poem for their partner, who really likes {topic}. 
    Ensure that the poem is custom and tailored to the client's partner and in a style that 
    is appropriate and conveys love, respect, kindness, and admiration""",
    expected_output=f"A lovely poem in a style that the client would like that incoporates the ideas and facts related to the {topic}",
    agent=manager,
)

In [ ]:
crew = Crew(
    agents = [manager, researcher, writer],
    tasks = [list_ideas, list_facts, write_poem, manager_task],
    process = Process.hierarchical,
    manager_llm=ChatOpenAI(temperature=0, model="gpt-4"),
)
result = crew.kickoff()

In [ ]:
print(result)

# 5. Outreach campaign manager

- Source: https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/9/tools-for-a-customer-outreach-campaign-(code)

In [ ]:
from crewai import Agent, Task, Crew, Process
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

# tools
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool, BaseTool

directory_read_tool = DirectoryReadTool(directory='./campaign-instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"
sentiment_analysis_tool = SentimentAnalysisTool()

# tasks
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

# crew
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)